In [1]:
import pandas as pd
import pyarrow as pa


# Read CSV with PyArrow 

## Convert to Pandas

In [3]:
import pandas as pd
import pyarrow.csv as pa_csv
from pandas import DataFrame


#  ignore line error
def csv_line_error(line: str) -> str:
    print(line)
    return "skip"


def set_column_type_string(filepath: str) -> dict:
    df: DataFrame = pd.read_csv(filepath, nrows=1, engine="python")
    column_type: dict = {}
    for col in df.columns.tolist():
        column_type[col] = pa.string()
    return column_type


def pyarrow_read_csv(filepath: str) -> DataFrame:
    # read csv file force type string
    # if not set columns big number is issue
    column_type = set_column_type_string(filepath)

    # ignore line error when read
    # call function csv_line_error when row invalid
    parse_options = pa_csv.ParseOptions(invalid_row_handler=csv_line_error)

    # ignore utf8
    # strings_can_be_null = True
    convert_options = pa_csv.ConvertOptions(check_utf8=False, column_types=column_type, strings_can_be_null=True)

    tbl: pa.Table = pa_csv.read_csv(filepath, parse_options=parse_options, convert_options=convert_options)
    df: DataFrame = tbl.to_pandas(types_mapper=pd.ArrowDtype)
    return df